In [ ]:
**1_keras_custom_classifier_with_transfer_learning_100_epochs.ipynb**
This version runs the model 100 epochs on the hands data which is sourced from my google drive.

<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-3/1-keras-custom-classifier-with-transfer-learning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-3/1-keras-custom-classifier-with-transfer-learning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This code is part of [Chapter 3: Cats versus Dogs: Transfer Learning in 30 Lines with Keras](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch03.html).

Note: In order to run this notebook on Google Colab you need to [follow these instructions](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb#scrollTo=WzIRIt9d2huC) so that the local data such as the images are available in your Google Drive.

# Building a Custom Classifier in Keras with Transfer Learning

As promised, it’s time to build our state of the art classifier in 30 lines or fewer! At a high level, we will follow the steps shown below:

- **Organize the data**: Download labeled images of cats and dogs from Kaggle. Then divide the images into training and validation folders.
- **Set up the configuration**: Define a pipeline for reading data, including preprocessing the images (e.g. resizing) and batching multiple images together.
- **Load and augment the data**: In the absence of a ton of training images, make small changes (augmentation) like rotation, zooming, etc to increase variation in training data.
- **Define the model**: Take a pre-trained model, remove the last few layers, and append a new classifier layer. Freeze the weights of original layers (i.e. make them unmodifiable). Select an optimizer algorithm and a metric to track (like accuracy).
- **Train and test**: Start training for a few iterations. Save the model to eventually load inside any application for predictions.

In [ ]:
!unzip train.zip -d data

## Organize the data

Before training, we need to store our [downloaded dataset](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip) in the right folder structure. Remember to make the `data` directory where we will be performing the refactoring. We’ll divide the images into two sets – training and validation. Our directory structure will look something like this: 

```
data
 |__train
 |    |__cat
 |    |__dog
 |__val
      |__cat
      |__dog
```

In Linux/Mac, the following lines of command can help achieve this directory structure:

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')


In [33]:
path_data = 'data-dogcat/'

In [22]:
#!ls -l /content/gdrive/My\ Drive/462/week\ 09/data

In [23]:
#!unzip train.zip
#%mv train data
#%cd data
#%mkdir train val
#%mkdir train/cat train/dog
#%mkdir val/cat val/dog

The 25,000 files inside the data folder are prefixed with ‘cat’ and ‘dog’. Now, move the files into their respective directories. To keep our initial experiment short, we’ll pick 250 random files per class and place them in training and validation folders. You can increase/decrease this number anytime, to experiment with a trade-off between accuracy and speed. 

Classification accuracy on previously unseen images (in the validation folder) is a good proxy for how the classifier would perform in the real world. Ideally, the more training images, the better the learning will be. And, the more validation images, the better our classifier would perform in the real-world.

In [ ]:
#!ls | grep cat | sort -R | head -250 | xargs -I {} mv {} train/cat/
#!ls | grep dog | sort -R | head -250 | xargs -I {} mv {} train/dog/
#!ls | grep cat | sort -R | head -250 | xargs -I {} mv {} val/cat/
#!ls | grep dog | sort -R | head -250 | xargs -I {} mv {} val/dog/

## Set up the configuration

Let's start off with our Python program and begin with importing the necessary packages.

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

Let's place all the configurations up-front. These can be modified in the future based on the dataset of your choice.

In [24]:
TRAIN_DATA_DIR = path_data + 'train/'
VALIDATION_DATA_DIR = path_data + 'val/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

## Load and augment the data

Colored images usually have 3 channels viz. red, green and blue, each with intensity value ranging from 0 to 255. To normalize it (i.e. bring the value between 0 to 1), we can rescale the image by dividing each pixel by 255. Or, we can use the default `preprocess_input` function in Keras which does the preprocessing for us.

In [27]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

Time to load the data from its directories and let the augmentation happen! 

A few key things to note:

- Training one image at a time can be pretty inefficient, so we can batch them into groups. 
- To introduce more randomness during the training process, we’ll keep shuffling the images in each batch.
- To bring reproducibility during multiple runs of the same program, we’ll give the random number generator a seed value.

In [15]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Now that the data is taken care of, we come to the most crucial component of our training process - the model. We will reuse a CNN previously trained on the ImageNet dataset, remove the ImageNet specific classifier in the last few layers, and replace it with our own classifier suited to our problem. For transfer learning, we’ll ‘freeze’ the weights of the original model, i.e. set those layers as unmodifiable, so only the layers of the new classifier (that we add) can be modified. To keep things fast, we’ll choose the MobileNet model. Don’t worry about the specific layers, we’ll dig deeper into those details in [Chapter 4](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch04.html).

## Define the model

In [16]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

## Train and test

With both the data and model ready, all we have left to do is train the model. This is also known as fitting the model to the data. For training any model, we need to pick a loss function, an optimizer, initial learning rate and a metric. Let's discuss these briefly:

- **Loss function**: The loss function is the objective being minimized. For example, in a task to predict house prices, the loss function could be the mean squared error.
- **Optimizer**: This is an optimization algorithm that helps minimize the loss function. We’ll choose `Adam`, one of the fastest optimizers out there.
- **Learning rate**: This defines how quickly or slowly you update the weights during training. Choosing an optimal learning rate is crucial - a big value can cause the training process to jump around, missing the target. On the other hand, a tiny value can cause the training process to take ages to reach the target. We’ll keep it at 0.001 for now.
- **Metric**: Choose a metric to judge the performance of the trained model. Accuracy is a good explainable metric, especially when the classes are not imbalanced, i.e. roughly equal in size. Note that this metric is not used during training to maximize or minimize an objective.

You might have noticed the term `epoch` here. One epoch means a full training step where the network has gone over the entire dataset.  One epoch may consist of several mini-batches.

Run this program and let the magic begin. If you don’t have a GPU, brew a cup of coffee while you wait. You’ll notice 4 statistics - `loss` and `accuracy` on both the training and validation data. You are rooting for the `val_acc`.

In [17]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [18]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/10
8/8 [==============================] - 355s 44s/step - loss: 0.6366 - acc: 0.7129 - val_loss: 0.0596 - val_acc: 0.9824
Epoch 2/10
8/8 [==============================] - 343s 43s/step - loss: 0.3087 - acc: 0.8668 - val_loss: 0.0295 - val_acc: 0.9922
Epoch 3/10
8/8 [==============================] - 349s 44s/step - loss: 0.2278 - acc: 0.9160 - val_loss: 0.0170 - val_acc: 0.9980
Epoch 4/10
8/8 [==============================] - 325s 41s/step - loss: 0.1842 - acc: 0.9365 - val_loss: 0.0137 - val_acc: 0.9980
Epoch 5/10
8/8 [==============================] - 327s 41s/step - loss: 0.1564 - acc: 0.9531 - val_loss: 0.0197 - val_acc: 0.9941
Epoch 6/10
8/8 [==============================] - 348s 43s/step - loss: 0.1575 - acc: 0.9385 - val_loss: 0.0138 - val_acc: 0.9961
Epoch 7/10
8/8 [==============================] - 360s 45s/step - loss: 0.1445 - acc: 0.9434 - val_loss: 0.0378 - val_acc: 0.9863
Epoch 8/10
8/8 [==============================] - 351s 44s/step - loss: 0.2174 - acc: 0.90

On our runs, it took was 5 seconds in the very first epoch to reach 90% accuracy on the validation set, with just 500 training images. Whoa! And by the 10th step, we observe about 97% validation accuracy. That’s the power of transfer learning. 

Without having the model previously trained on ImageNet, getting a decent accuracy on this task would have taken (1) training time anywhere between a couple of hours to a few days (2) tons of more data to get decent results.

Before we forget, save the model you trained.

In [28]:
model.save('model.h5')

## Model Prediction

Now that you have a trained model, you might eventually want to use it later for your application. We can now load this model anytime and classify an image. The Keras function `load_model`, as the name suggests loads the model. 

In [29]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

Now let’s try loading our original sample images and see what results we get.

In [34]:
A538.jpg

img_path = path_data + 'dog.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[0.13143045 0.86856955]]
{'cat': 0, 'dog': 1}


In [ ]:
Hands

In [37]:
path_data = 'data-asl/'

In [42]:
TRAIN_DATA_DIR = path_data + 'train/'
VALIDATION_DATA_DIR = path_data + 'val/'
TRAIN_SAMPLES = 180
VALIDATION_SAMPLES = 20
NUM_CLASSES = 26
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [43]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [44]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 4680 images belonging to 26 classes.
Found 520 images belonging to 26 classes.


In [45]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [46]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

Epoch 1/10
3/3 [==============================] - 117s 39s/step - loss: 3.7981 - acc: 0.0312 - val_loss: 3.7050 - val_acc: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 110s 37s/step - loss: 3.3011 - acc: 0.0677 - val_loss: 3.4315 - val_acc: 0.0000e+00
Epoch 3/10
3/3 [==============================] - 111s 37s/step - loss: 3.2329 - acc: 0.0573 - val_loss: 3.3363 - val_acc: 0.0000e+00
Epoch 4/10
3/3 [==============================] - 108s 36s/step - loss: 3.1954 - acc: 0.0781 - val_loss: 3.2801 - val_acc: 0.0000e+00
Epoch 5/10
3/3 [==============================] - 107s 36s/step - loss: 3.1735 - acc: 0.0729 - val_loss: 3.2045 - val_acc: 0.0000e+00
Epoch 6/10
3/3 [==============================] - 117s 39s/step - loss: 3.1245 - acc: 0.1198 - val_loss: 3.1331 - val_acc: 0.0000e+00
Epoch 7/10
3/3 [==============================] - 114s 38s/step - loss: 3.1530 - acc: 0.0573 - val_loss: 3.1769 - val_acc: 0.0000e+00
Epoch 8/10
3/3 [==============================] - 112s 37s/ste

In [47]:
model.save('model.h5')

In [48]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

In [49]:
img_path = path_data + 'A538.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)
val

[[0.06265437 0.05322449 0.0519603  0.02691153 0.03446881 0.02527515
  0.03545844 0.08238315 0.02727935 0.02183942 0.02353981 0.02773061
  0.02001667 0.03709147 0.05710829 0.06187703 0.05511278 0.07396112
  0.03585918 0.03234247 0.03859612 0.01822748 0.03009981 0.0203944
  0.0193372  0.02725051]]
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
